In [1]:
import os, sys
sys.path.insert(0, os.environ['PROJECT_PATH'])

In [2]:
import pandas as pd, requests, datetime

In [3]:
from config.resources import path_to

In [4]:
funding_rounds_df = pd.read_csv(path_to['csv_export'].format('funding_rounds'), encoding='latin1')
funding_rounds_df['announced_on'] = pd.to_datetime(funding_rounds_df['announced_on'])

funding_rounds_df.columns

Index([u'company_name', u'country_code', u'state_code', u'region', u'city',
       u'investment_type', u'announced_on', u'raised_amount_usd',
       u'raised_amount', u'raised_amount_currency_code',
       u'post_money_valuation_usd', u'post_money_valuation',
       u'post_money_currency_code', u'investor_count', u'cb_url',
       u'company_uuid', u'funding_round_uuid', u'created_at', u'updated_at',
       u'investor_names', u'investor_uuids'],
      dtype='object')

In [5]:
curr_year = datetime.datetime.now().year
year_range = list(map(str, range(curr_year-2, curr_year+1)))

In [6]:
funding_rounds_us_df = funding_rounds_df[ funding_rounds_df['country_code'] == 'USA' ]
funding_rounds_us_df = funding_rounds_us_df[ funding_rounds_us_df['announced_on'].dt.year.isin(year_range) ]

print(funding_rounds_us_df.shape)

funding_rounds_us_df.head()

(21362, 21)


,company_name,country_code,state_code,region,city,investment_type,announced_on,raised_amount_usd,raised_amount,raised_amount_currency_code,...,post_money_valuation,post_money_currency_code,investor_count,cb_url,company_uuid,funding_round_uuid,created_at,updated_at,investor_names,investor_uuids
4,HaloSource,USA,WA,Seattle,Bothell,post_ipo_equity,2017-07-19,2200000.0,NaN,NaN,...,NaN,NaN,NaN,https://www.crunchbase.com/funding-round/halos...,e2abf2a9-143f-08fc-471c-ef13762553ad,00025001-1cf4-8edb-abc3-05a0bb0a3a63,2017-07-20 05:09:17,2017-07-20 05:28:02,{},{}
11,Carbon Robotics,USA,CA,SF Bay Area,San Francisco,seed,2016-12-07,3240354.0,NaN,NaN,...,NaN,NaN,1.0,https://www.crunchbase.com/funding-round/katia,dcfb7bb7-fecf-7916-43a7-30b1c153dc5d,0003c42b-498e-cd53-aec6-53461d667c79,2016-09-15 11:47:07,2017-11-17 11:31:40,{},{}
14,Tandem Diabetes Care,USA,CA,San Diego,San Diego,post_ipo_equity,2017-03-29,23100000.0,NaN,NaN,...,NaN,NaN,NaN,https://www.crunchbase.com/funding-round/tande...,1d737422-4985-7901-4aec-8c8af63f3570,00044f9f-512a-a9f5-de44-a74f665c8bc1,2017-03-30 04:42:40,2017-03-30 04:44:26,{},{}
30,Shelvspace,USA,AZ,Phoenix,Scottsdale,series_a,2017-03-03,1000000.0,NaN,NaN,...,NaN,NaN,1.0,https://www.crunchbase.com/funding-round/shelv...,e874c539-d575-1ef3-321b-23268f704da7,000b93c4-3022-4c3f-8301-9c40b66c51d6,2017-11-15 04:17:22,2017-11-15 17:38:22,{},{}
35,New York Medical College,USA,NY,NY - Other,Valhalla,grant,2017-12-28,1250000.0,NaN,NaN,...,NaN,NaN,2.0,https://www.crunchbase.com/funding-round/new-y...,06aadb03-aa45-4ff7-9b17-a9c07336117b,000c585e-d8ed-4c44-9daf-0a4cd7914463,2017-12-30 04:06:04,2017-12-30 04:06:04,"{""Empire State Development"",""NY Regional Econo...","{5a554fe0-2241-a57e-27b7-1c04ba844cc7,c5ae34b7..."


In [ ]:
from src.data.msa_mapper.map_loc_to_msa import MSAMapper

loc_columns = [ 'city', 'state_code', 'country_code' ]
fr_us_locs_df = funding_rounds_us_df[loc_columns]

msa_mapping_client = MSAMapper(fr_us_locs_df.fillna(''))
fr_us_with_msa_df = msa_mapping_client.map_data()